In [1]:
import cv2

# Open webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

for i in range(2):  # Capture two images
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            break
        
        # Show live feed
        cv2.imshow("Press 's' to capture", frame)
        
        # Wait for key press
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):  # Press 's' to save the image
            filename = f"image{i+1}.jpg"
            cv2.imwrite(filename, frame)
            print(f"Saved {filename}")
            break
        elif key == ord('q'):  # Press 'q' to quit
            cap.release()
            cv2.destroyAllWindows()
            exit()

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
# !pip uninstall opencv-python
# !pip install opencv-python


In [3]:
import numpy as np

# Manually define corresponding points in both images
pts_src = np.array([[100, 200], [400, 200], [100, 400], [400, 400]])  # Image 1 points
pts_dst = np.array([[120, 220], [420, 210], [130, 420], [430, 430]])  # Image 2 points

# Compute Homography
H, status = cv2.findHomography(pts_src, pts_dst)

print("Homography Matrix:\n", H)


Homography Matrix:
 [[ 8.49335303e-01  5.05908419e-02  2.17577548e+01]
 [-9.44854751e-02  9.79615953e-01  2.76760217e+01]
 [-2.95420975e-04  1.47710487e-05  1.00000000e+00]]


In [1]:
# Load the first image
img1 = cv2.imread("image1.jpg")

# Get image dimensions
height, width, _ = img1.shape

# Warp perspective using homography matrix
warped_img = cv2.warpPerspective(img1, H, (width, height))

# Display original and warped images
cv2.imshow("Original Image", img1)
cv2.imshow("Warped Image", warped_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/gabby/.local/lib/python3.10/site-packages/cv2/qt/plugins"


Homography Matrix:
 [[ 2.14978557e+00 -1.24258615e-01 -2.93516482e+02]
 [ 3.84753182e-01  1.47942642e+00 -1.91532314e+02]
 [ 1.11401044e-03 -2.25946588e-04  1.00000000e+00]]


In [ ]:
import cv2
import numpy as np

# Load images
img1 = cv2.imread("image1.jpg", cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread("image2.jpg", cv2.IMREAD_GRAYSCALE)

# Initialize ORB detector
orb = cv2.ORB_create()

# Detect keypoints and descriptors
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

# Use BFMatcher with Hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)

# Sort matches by distance (lower is better)
matches = sorted(matches, key=lambda x: x.distance)

# Draw top matches
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None, flags=2)
cv2.imshow("Feature Matches", img_matches)
cv2.waitKey(0)

# Extract point coordinates
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

# Compute Homography
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

print("Homography Matrix:\n", H)

# Warp image
height, width = img2.shape
warped_img = cv2.warpPerspective(img1, H, (width, height))

# Show result
cv2.imshow("Warped Image", warped_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
